<a href="https://colab.research.google.com/github/ppiont/tensor-flow-state/blob/master/data_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
%cd '/gdrive/My Drive/tensor-flow-state/tensor-flow-state'

/gdrive/My Drive/tensor-flow-state/tensor-flow-state


In [0]:
sensor_list = ['RWS01_MONIBAS_0021hrl0403ra.csv', 'RWS01_MONIBAS_0021hrl0409ra.csv', 'RWS01_MONIBAS_0021hrl0414ra.csv', 'RWS01_MONIBAS_0021hrl0420ra.csv', 'RWS01_MONIBAS_0021hrl0426ra.csv']

In [0]:
import datetime
def dateparse (time_in_secs):    
    return datetime.datetime.fromtimestamp(float(time_in_secs))

import pandas as pd
def repair_datetime_index(df, freq = 'T'):
    df = df.loc[~df.index.duplicated(keep='first')]
    df = df.reindex(pd.date_range(start = df.index.min(), end = df.index.max(), freq = freq))
    return df

import numpy as np
def convert_invalid_to_nans(df):
    df.loc[df.flow < 0] = np.nan
    df.loc[df.speed == -2] = np.nan
    return df

import os
def reduce_cols(sensors, path_in = "data/ndw_raw/", path_out = "data/"):
    for sensor in sensors:
        df = pd.read_csv(os.path.join(path_in, sensor), header = None, \
                         usecols = [0, 86, 87], names = ['timestamp', 'speed', 'flow'], \
                         index_col = 'timestamp', parse_dates = True, date_parser = dateparse)
        df.flow /= 60
        df = repair_datetime_index(df)
        df = convert_invalid_to_nans(df)
        df.to_csv(path_out + sensor)


In [0]:
reduce_cols(sensor_list)